In [0]:
# Full tutorial with explanation at https://medium.com/@johnson.doc/create-a-world-class-deep-learning-model-using-google-drive-and-deploy-it-as-an-app-7edc6695bbac 
# Installing TuriCreate. This is our main package. TuriCreate is open sourced. 
# You can get more info about TuriCreate at https://github.com/apple/turicreate

!pip install turicreate

  Using cached mxnet-0.12.1-py2.py3-none-manylinux1_x86_64.whl
  Found existing installation: mxnet 1.1.0
    Uninstalling mxnet-1.1.0:
      Successfully uninstalled mxnet-1.1.0


In [0]:
# Finding the directory we are in
!ls

datalab  drive


Next code block will mount your google drive. This will help you to access files stored in your google drive. 
Druing installation you see a prompt with a link -Please, open the following URL in a web browser: https://accounts.google.com/o ....
Click the link. Copy and paste the code from the link in the form field and press enter. 
Sometime You may have to run this code block twice. I think this is a bug in colab. 

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmple_z39tn/pubring.gpg' created
gpg: /tmp/tmple_z39tn/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
# Mount your google drive to access files from it 
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
# The image folder that I am using for this tutorial is located in my google drive under 'app' folder
# So we need to change directory to the 'app' folder

import os
os.chdir("drive/app")

In [0]:
# make sure that we are in the right folder 
!ls

cats-dogs.sframe  ImageFolder


In [0]:
# importing turicreate as tc

import turicreate as tc

In [0]:
# Loading the images from ImageFolder in my google drive. This 'ImageFolder' has 2 folders
# 'Cat' and 'Dog'
# Each folder has about 1000 images
# Original images were obtained from https://www.kaggle.com/c/dogs-vs-cats/data
# First the whole data set from kaggle website was downloaded to my desktop. 
# After that I created a folder called 'ImageFolder' in my desktop with sub folders 'Cat' & 'Dog'
# Then I copied the first 1000 images of cats and dogs to the individual folders
# Later the 'ImageFolder' was uploaded to the 'app' folder in my google drive

data = tc.image_analysis.load_images('ImageFolder', with_path=True)

Read 1146 images in 5.00121 secs	speed: 229.145 file/sec

It took 5 seconds to process the files. 

In [0]:
# Labelling each images in the SFrame using the folder name

data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

In [0]:
#Displaying the first 5 rows in data to make sure that we got everything right 

data.head()

path,image,label
/content/drive/app/ImageFolder/Cat/cat.34.jpg ...,Height: 499 Width: 375,cat
/content/drive/app/ImageFolder/Dog/dog.837.jpg ...,Height: 400 Width: 419,dog
/content/drive/app/ImageFolder/Dog/dog.356.jpg ...,Height: 375 Width: 499,dog
/content/drive/app/ImageFolder/Dog/dog.215.jpg ...,Height: 500 Width: 349,dog
/content/drive/app/ImageFolder/Dog/dog.804.jpg ...,Height: 287 Width: 301,dog
/content/drive/app/ImageFolder/Dog/dog.393.jpg ...,Height: 375 Width: 499,dog
/content/drive/app/ImageFolder/Cat/cat.812.jpg ...,Height: 375 Width: 499,cat
/content/drive/app/ImageFolder/Cat/cat.594.jpg ...,Height: 499 Width: 375,cat
/content/drive/app/ImageFolder/Dog/dog.315.jpg ...,Height: 431 Width: 362,dog
/content/drive/app/ImageFolder/Cat/cat.130.jpg ...,Height: 500 Width: 422,cat


In [0]:
# Saving the data frame to our google drive 
data.save('cats-dogs.sframe')

In [0]:
# Splitting the image data set into training (80%) and testing (20%) data
train_data, test_data = data.random_split(0.8)

In [0]:
# Switching to the root directory in our google virtual machine
os.chdir("/usr")

In [0]:
# installing other packahes to use GPU
!apt install libnvrtc8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libnvrtc8.0 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [0]:
# uninstalling previosu versions of mxnet
!pip uninstall -y mxnet

Uninstalling mxnet-0.12.1:
  Successfully uninstalled mxnet-0.12.1


In [0]:
# Installing the latest version of mxnet to work with turi create
!pip install mxnet==1.1.0

  Using cached mxnet-1.1.0-py2.py3-none-manylinux1_x86_64.whl


In [0]:
# making sure that mxnet is working
# If you get an erroe here, you need to fix it before going forward
import mxnet as m2

In [0]:
# Automatically pick the right model based on your data.
# Note: Because the dataset is large, model creation may take hours.

model = tc.image_classifier.create(train_data, target='label')

Resizing images...
Performing feature extraction on resized images...
Completed  512/1591
Completed 1024/1591
Completed 1536/1591
Completed 1591/1591
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) '__image_features__' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 6         | 10       | 0.500000  | 2.880737     | 0.992110          | 1.000000            |

| 10        | 15       | 1.000000  | 3.989484     | 0.998028          | 1.000000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Took only about 7 minutes to do this with GPU!

In [0]:
# storing the prediction 

predictions = model.predict(test_data)

In [0]:
# Evaluating the model 

metrics = model.evaluate(test_data)
print(metrics['accuracy'])

0.9902676399026764


In [0]:
# Printing confusion matrix

print ("Confusion Matrix : \n%s" % metrics['confusion_matrix'])

Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|     dog      |       cat       |   2   |
|     cat      |       dog       |   2   |
|     cat      |       cat       |  195  |
|     dog      |       dog       |  212  |
+--------------+-----------------+-------+
[4 rows x 3 columns]



In [0]:
# Save the model for later use in Turi Create
model.save('mymodel.model')

In [0]:
# Export for use in Core ML
model.export_coreml('MyCustomImageClassifier.mlmodel')

In [0]:
!ls

MyCustomImageClassifier.mlmodel  games	  lib64-nvidia	 mymodel.txt	 sbin
bin				 include  local		 mymodel2.h5	 share
cats-dogs.sframe		 lib	  mymodel.model  mymodel2.model  src


In [0]:
# Downloading the model for use in Xcode

from google.colab import files

files.download('MyCustomImageClassifier.mlmodel')